# Control theory and Vicsek Model - Clustering analysis : dynamics of clusters over time, graph theory applied to clustering

## Importing modules and libraries 

In [90]:
# General imports
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pandas as pd
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from IPython.display import Video, display
from collections import Counter

In [91]:
# Module imports
import models.vicsek as vicsek
import visualisation as visualisation
import utils
import animation.Animator2D as Animator2D
import animation.MatplotlibAnimator as MatplotlibAnimator

## Example of simulation 

In [92]:
# Initialize the Vicsek model.
radius = 1
L = 50
N = 300

In [93]:
simulator = vicsek.Vicsek(numberOfParticles=N, domainSize=(L, L), radius=radius)
# Simulate the Vicsek model.
simulationData = simulator.simulate()

In [94]:
df = utils.simulationDataToDataframe(simulationData)
df.to_csv('data/vicsek.csv', index=False)

In [95]:
df_labels = utils.clusters_over_time(df, k_coef=1.8, L=L, min_samples=5)
df_labels.to_csv('data/vicsek_labels.csv', index=False)

In [ ]:
df_labels.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-1,0,1,-1,2,-1,3,-1,-1,1,...,-1,10,-1,9,5,-1,2,9,14,3
1,-1,0,1,-1,2,-1,3,5,19,1,...,-1,16,-1,17,7,-1,2,17,-1,3
2,-1,0,1,-1,2,-1,14,-1,-1,1,...,-1,12,-1,17,5,-1,2,17,-1,14
3,-1,19,0,-1,1,-1,2,-1,-1,3,...,-1,13,3,18,14,-1,1,18,-1,2
4,-1,19,-1,-1,0,14,1,-1,-1,2,...,-1,11,2,15,12,-1,0,15,-1,1


## Plot the cluster network with `python-igraph`

https://python.igraph.org/en/stable/ c pas ouf ouf 

In [ ]:
class Flock: 
    id_counter = 0

    def __init__(self, label):
        self.id = Flock.id_counter
        self.bird_list = []
        self.size = 0
        self.aliveBool = True
        self.label = label
        Flock.id_counter += 1
    
    def add_bird(self, i):
        self.bird_list.append(i)
        self.size += 1
    
    def remove_bird(self, i):
        self.bird_list.remove(i)
        self.size -= 1
    
        
    
    def get_size(self):
        return self.size
    
    def get_birds(self):
        return self.bird_list
    
    def get_id(self):
        return self.id
    
    def kill(self):
        if self.size == 0:
            self.aliveBool = False
            return not self.aliveBool
        else :
            return not self.aliveBool
    
    def is_in_flock(self, i):
        return i in self.bird_list
    
    def merge(self, flock):
        for bird in flock.get_birds():
            self.add_bird(bird)
            flock.remove_bird(bird)
        flock.kill()
    
    def split(self, new_flock, birdlist):
        for bird in birdlist:
            new_flock.add_bird(bird)
            self.remove_bird(bird)
        return new_flock
    
    def get_label(self):
        return self.label
    
    def set_label(self, label):
        self.label = label
        
    def __str__(self) -> str:
        ret = f"Flock {self.id} with {self.size} birds \n"
        ret += f"Label : {self.label} \n"
        ret += f"Bird List : {self.bird_list}"
        return ret
    
    def reset_id_counter():
        Flock.id_counter = 0
    


In [ ]:
# history of new labels 
new_labels = np.zeros((len(df_labels), N)) - 1
new_labels.shape

(1001, 300)

In [ ]:
previous_label = df_labels.iloc[0].to_numpy()
next_label = df_labels.iloc[1].to_numpy()
flocks = []

Flock.reset_id_counter()

# Create the initial flocks
for lbl in np.sort(list(set(previous_label))):
    if lbl != -1:
        flocks.append(Flock(lbl))

# update flocks 
for flock in flocks : 
    lbl = flock.get_label()
    for bird, bird_label in enumerate(previous_label):
        if bird_label == lbl:
            flock.add_bird(bird)
    print(flock)
# update new labels
new_labels[0] = previous_label

Flock 0 with 18 birds 
Label : 0 
Bird List : [1, 25, 26, 39, 60, 71, 124, 142, 177, 183, 193, 194, 203, 208, 251, 265, 277, 287]
Flock 1 with 35 birds 
Label : 1 
Bird List : [2, 9, 14, 18, 30, 40, 44, 50, 58, 69, 77, 86, 96, 99, 107, 113, 121, 128, 131, 141, 143, 146, 150, 151, 176, 190, 197, 222, 232, 235, 245, 254, 266, 281, 289]
Flock 2 with 11 birds 
Label : 2 
Bird List : [4, 48, 154, 160, 165, 205, 253, 269, 279, 285, 296]
Flock 3 with 7 birds 
Label : 3 
Bird List : [6, 35, 53, 83, 167, 276, 299]
Flock 4 with 15 birds 
Label : 4 
Bird List : [13, 15, 16, 46, 102, 104, 106, 120, 139, 182, 198, 204, 213, 214, 272]
Flock 5 with 17 birds 
Label : 5 
Bird List : [24, 49, 115, 117, 133, 149, 157, 188, 192, 212, 230, 233, 257, 262, 270, 273, 294]
Flock 6 with 5 birds 
Label : 6 
Bird List : [28, 67, 147, 166, 170]
Flock 7 with 5 birds 
Label : 7 
Bird List : [29, 185, 187, 239, 271]
Flock 8 with 7 birds 
Label : 8 
Bird List : [33, 57, 103, 132, 148, 195, 220]
Flock 9 with 17 birds 


In [ ]:
# update flocks at next iteration 
DEBUG = True
logs = ""
for flock in flocks : 
    # get the label of the flock at the previous iteration
    previous_lbl = flock.get_label()
    # get the list of birds 
    bird_list = flock.get_birds()
    # get their new labels 
    next_flock_labels = next_label[bird_list]

        
    # print([[lab, next_flock_labels.count(lab)] for lab in set(next_flock_labels)])
    # Count birds with each new labels
    count = Counter(next_flock_labels)
    biggest = count.most_common(1)
    logs += f"Label {flock.get_label()} : {count}\n"

    if biggest[0][0] == -1 and len(count) > 1: # If the most common label is noise (i.e. flock is split) and there is more than one label
        biggest = count.most_common(2)[1]
    elif biggest[0][0] == -1 and len(count) == 1: # If the most common label is noise (i.e. flock is split) and there is only one label the flock dies
        # All the birds became noise : kill the flock
        # Kill the flock 
        flock.kill()
        if DEBUG : 
            print(count)
            logs += f"Flock {flock.get_id()} was killed\n"
            print(f"Flock {flock.get_id()} was killed")
        flocks.pop(flocks.index(flock))
        pass 
    else : # If the most common label is not noise (i.e. no major dispersion of particles)
        print("SPlit! ")

    # New label 
    new_lbl = biggest[0][0]
    if DEBUG : 
        print("# =========================== #")
        print("# === State before update === #")
        print("# =========================== #")
        print(flock)
        print(f'Counter {count}')
        print(f'Most common label {biggest}')
        print("# =========================== #\n")

print(logs)

SPlit! 
# =========================== #
# === State before update === #
# =========================== #
Flock 0 with 18 birds 
Label : 0 
Bird List : [1, 25, 26, 39, 60, 71, 124, 142, 177, 183, 193, 194, 203, 208, 251, 265, 277, 287]
Counter Counter({0: 11, 13: 5, -1: 2})
Most common label [(0, 11)]
# =========================== #

SPlit! 
# =========================== #
# === State before update === #
# =========================== #
Flock 1 with 35 birds 
Label : 1 
Bird List : [2, 9, 14, 18, 30, 40, 44, 50, 58, 69, 77, 86, 96, 99, 107, 113, 121, 128, 131, 141, 143, 146, 150, 151, 176, 190, 197, 222, 232, 235, 245, 254, 266, 281, 289]
Counter Counter({1: 11, 9: 11, 20: 5, -1: 4, 19: 3, 14: 1})
Most common label [(1, 11)]
# =========================== #

SPlit! 
# =========================== #
# === State before update === #
# =========================== #
Flock 2 with 11 birds 
Label : 2 
Bird List : [4, 48, 154, 160, 165, 205, 253, 269, 279, 285, 296]
Counter Counter({2: 9, -1: 2})
M